In [ ]:
!pip install streamlit pyngrok pandas matplotlib plotly flask


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.9/41.9 kB 1.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.7/8.7 MB 40.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 22.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.3/79.3 kB 4.3 MB/s eta 0:00:00


In [ ]:
%%writefile app.py
import streamlit as st
import pandas as pd
import random
import plotly.express as px

def generate_data():
    return {
        'timestamp': pd.Timestamp.now(),
        'temperature': random.uniform(20.0, 30.0),
        'humidity': random.uniform(30.0, 70.0),
        'soil_moisture': random.uniform(300, 700),
        'CO2': random.uniform(400, 5000)
    }

data = [generate_data() for _ in range(100)]
df = pd.DataFrame(data)

# Traffic light system for soil moisture
def traffic_light(moisture):
    if moisture < 350:
        return 'red'
    elif 350 <= moisture <= 450:
        return 'yellow'
    else:
        return 'green'

df['traffic_light'] = df['soil_moisture'].apply(traffic_light)

st.title('EnviroMonitor Dashboard')

# Summary Statistics
st.write("### Summary Statistics")

col1, col2, col3 = st.columns(3)
col1.metric("Min Temp", f"{df['temperature'].min():.2f} °C")
col2.metric("Max Temp", f"{df['temperature'].max():.2f} °C")
col3.metric("Avg Temp", f"{df['temperature'].mean():.2f} °C")

col1.metric("Min Humidity", f"{df['humidity'].min():.2f} %")
col2.metric("Max Humidity", f"{df['humidity'].max():.2f} %")
col3.metric("Avg Humidity", f"{df['humidity'].mean():.2f} %")

col1.metric("Min Soil Moisture", f"{df['soil_moisture'].min():.2f}")
col2.metric("Max Soil Moisture", f"{df['soil_moisture'].max():.2f}")
col3.metric("Avg Soil Moisture", f"{df['soil_moisture'].mean():.2f}")

col1.metric("Min CO2", f"{df['CO2'].min():.2f} ppm")
col2.metric("Max CO2", f"{df['CO2'].max():.2f} ppm")
col3.metric("Avg CO2", f"{df['CO2'].mean():.2f} ppm")

# Visualizations
st.write("### Monthly Avg Temp & Avg of Avg Barometer (in)")
fig = px.bar(df, x='timestamp', y=['temperature', 'CO2'], barmode='group')
st.plotly_chart(fig)

st.write("### Sum Avg Dewpoint (°F) & Avg Humidity (%) by Avg Windspeed (MPH)")
fig = px.scatter(df, x='humidity', y='temperature', color='CO2', size='soil_moisture', hover_data=['timestamp'])
st.plotly_chart(fig)

st.write("### Yearly Avg of Max Heat Index (°F)")
fig = px.treemap(df, path=[px.Constant("all"), 'timestamp'], values='temperature')
st.plotly_chart(fig)

st.write("### Avg of Diff_Pressure by Month")
fig = px.bar(df, x='temperature', y='timestamp', orientation='h')
st.plotly_chart(fig)

st.write("### Max Gust Speed (MPH) Avg")
fig = px.pie(df, values='CO2', names='timestamp')
st.plotly_chart(fig)

st.write("### Yearly Rainfall Sum for Month")
fig = px.bar(df, x='timestamp', y='humidity', color='temperature', labels={'humidity': 'Humidity (%)'})
st.plotly_chart(fig)

# Traffic Light Visualization
st.write("### Soil Moisture Traffic Light")
fig = px.scatter(df, x='timestamp', y='soil_moisture', color='traffic_light', size='soil_moisture', color_discrete_map={'red':'red', 'yellow':'yellow', 'green':'green'})
st.plotly_chart(fig)


Overwriting app.py


In [ ]:
%%writefile server.py
from flask import Flask, request, jsonify
import sqlite3
from datetime import datetime

app = Flask(__name__)

def init_db():
    conn = sqlite3.connect('data.db')
    c = conn.cursor()
    c.execute('''CREATE TABLE IF NOT EXISTS sensor_data
                 (timestamp TEXT, temperature REAL, humidity REAL, soil_moisture REAL, CO2 REAL)''')
    conn.commit()
    conn.close()

@app.route('/data', methods=['POST'])
def receive_data():
    data = request.json
    conn = sqlite3.connect('data.db')
    c = conn.cursor()
    c.execute("INSERT INTO sensor_data VALUES (?, ?, ?, ?, ?)",
              (datetime.now(), data['temperature'], data['humidity'], data['soil_moisture'], data['CO2']))
    conn.commit()
    conn.close()
    return jsonify({'status': 'Data received'}), 200

if __name__ == '__main__':
    init_db()
    app.run(debug=False)  # Disable debug mode


Overwriting server.py


In [ ]:
from pyngrok import ngrok

# Option 1: Set authtoken globally
ngrok.set_auth_token("2o4xXT5hrPaGkeCYQheHKUK94FL_4bhGHikHRKKbbmiAoMK3Y")


In [ ]:
!python server.py &
flask_tunnel = ngrok.connect(5000)
print("Flask server running on:", flask_tunnel.public_url)


 * Serving Flask app 'server'
 * Debug mode: off
Address already in use
Port 5000 is in use by another program. Either identify and stop that program, or start the server with a different port.
Flask server running on: https://34f1-34-34-13-43.ngrok-free.app


In [ ]:

streamlit_tunnel = ngrok.connect(8501)
print("Streamlit app running on:", streamlit_tunnel.public_url)
!streamlit run app.py &


Streamlit app running on: https://c3a0-34-34-13-43.ngrok-free.app



  You can now view your Streamlit app in your browser.

  Local URL: http://localhost:8501
  Network URL: http://172.28.0.12:8501
  External URL: http://34.34.13.43:8501

